# sm'

In [ ]:
//// test

open testing

In [ ]:
open rust
open rust_operators
open sm'_real

## rust

### std_string

In [ ]:
//// real

nominal std_string =
    `(
        backend_switch `(()) `({}) {
            Fsharp =
                (fun () =>
                    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::string::String\")>]\ntype std_string_String = class end\n\#else\ntype std_string_String = string\n#endif\n"
                ) : () -> ()
        }
        $'' : $'std_string_String'
    )

In [ ]:
type std_string = sm'_real.std_string

### to_string'

In [ ]:
inl to_string' forall t. (x : t) : std_string =
    !\($'$"!x.to_string()"')

### from_std_string

In [ ]:
//// real

inl from_std_string (str : std_string) : string =
    open rust
    rust.emit_expr `std_string `string str ($'"fable_library_rust::String_::fromString($0)"' : string)

In [ ]:
inl from_std_string (str : std_string) : string =
    real sm'_real.from_std_string str

## sm'

### symbol_to_string

In [ ]:
//// real

inl symbol_to_string forall t {symbol}. : string =
    // inl x = real_core.type_lit_to_lit `t
    // inl x = real_core.type_to_symbol `t
    // inl x = real_core.type_lit_to_lit `t
    // !!!!SymbolToString (`(`t))
    inl x = real_core.type_to_symbol `t
    !!!!SymbolToString (x)

In [ ]:
inl symbol_to_string forall t {symbol}. (x : t) : string =
    real symbol_to_string `t

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda

.test
|> symbol_to_string
|> _assert_eq "test"

.py output (Python):
{ name = __assert_eq; expected = test }

.gleam output (Gleam):
{ name = __assert_eq; expected = test }



.fsx output:
{ name = __assert_eq; expected = test }


In [ ]:
//// test
//// real
///! gleam
///! fsharp
///! cuda

open testing
inl x = .test
inl x = symbol_to_string `(`x)
_assert_eq `string "test" x

.py output (Python):
{ name = __assert_eq; expected = test }

.gleam output (Gleam):
{ name = __assert_eq; expected = test }



.fsx output:
{ name = __assert_eq; expected = test }


### index

In [ ]:
inl index i (str : string) : char =
    sm.index str i

### length

In [ ]:
inl length forall dim {int}. (input : string) : dim =
    input |> sm.length

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda

"abc"
|> length
|> _assert_eq 3i32

.py output (Python):
{ name = __assert_eq; expected = 3 }

.gleam output (Gleam):
{ name = __assert_eq; expected = 3 }



.fsx output:
{ name = __assert_eq; expected = 3 }


### to_char_list

In [ ]:
inl to_char_list (str : string) : list char =
    listm.init (str |> length) (fun (i : i64) => str |> index i)

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda

"abc"
|> to_char_list
|> _assert_eq [ 'a'; 'b'; 'c' ]

.py output (Python):
{ name = __assert_eq; expected = UH0_1(v0='a', v1=UH0_1(v0='b', v1=UH0_1(v0='c', v1=UH0_0()))) }

.gleam output (Gleam):
{ name = __assert_eq; expected = Uh0i1("a", Uh0i1("b", Uh0i1("c", Uh0i0))) }



.fsx output:
{ name = __assert_eq; expected = UH0_1 ('a', UH0_1 ('b', UH0_1 ('c', UH0_0))) }


### is_empty

In [ ]:
inl is_empty (input : string) : bool =
    length input = 0i32

### to_option

In [ ]:
inl to_option s =
    if s |> is_empty
    then None
    else s |> Some

### slice

In [ ]:
inl slice forall t {number; int}. (from : t) (to : t) s : string =
    backend_switch {
        Gleam = fun () => sm.slice s { from to } : string
        Fsharp = fun () => sm.slice s { from to } : string
        Python = fun () => sm.slice s { from to = if var_is s || var_is to then to + 1 else to } : string
    }

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda

"abcdef"
|> slice 1i32 3i32
|> _assert_eq "bcd"

(join "abcde")
|> slice 1i32 3i32
|> _assert_eq "bcd"

"abcde"
|> slice 1i32 (join 3i32)
|> _assert_eq "bcd"

(join "abcde")
|> slice 1i32 (join 3i32)
|> _assert_eq "bcd"


.py output (Python):
{ name = __assert_eq; expected = bcd }
{ name = __assert_eq; expected = bcd }
{ name = __assert_eq; expected = bcd }
{ name = __assert_eq; expected = bcd }


.gleam output (Gleam):
{ name = __assert_eq; expected = bcd }
{ name = __assert_eq; expected = bcd }
{ name = __assert_eq; expected = bcd }
{ name = __assert_eq; expected = bcd }




.fsx output:
{ name = __assert_eq; expected = bcd }
{ name = __assert_eq; expected = bcd }
{ name = __assert_eq; expected = bcd }
{ name = __assert_eq; expected = bcd }


### append_real

In [ ]:
//// real

inl append_real (x : string) (s : string) : string =
    backend_switch `string `({}) {
        Gleam = (fun () => $'!s <> !x ' : string) : () -> string
        Fsharp = (fun () => (+.) `string s x) : () -> string
        Python = (fun () => (+.) `string s x) : () -> string
    }

### append

In [ ]:
inl append (x : string) (s : string) : string =
    real append_real x s

### (++#)

In [ ]:
//// real

inl (++#) (a : string) (b : string) : string =
    append_real b a

In [ ]:
inl (++#) (a : string) (b : string) : string =
    real (++#) a b

### (++\\#)

In [ ]:
inl (++\\#) a b =
    a ++# "\\n" ++# b

### (++\#)

In [ ]:
inl (++\#) a b =
    a ++# "\n" ++# b

### format_debug

In [ ]:
//// real

inl format_debug forall t. (x : t) : string =
    typecase t with
    | string => x
    | _ =>
        backend_switch `string `({}) {
            Gleam = (fun () =>
                typecase t with
                | char => to `t `string x
                | _ =>
                    global "import gleam/string"
                    inl x =
                        typecase t with
                        | array_base _ => $'!x |> array.to_list' : t
                        | _ => x
                    $'string.inspect(!x)' : string
                    ) : () -> string
            Fsharp = (fun () => $'$"%A{!x}"' : string) : () -> string
            Python = (fun () => $'f"{!x}"' : string) : () -> string
        }

In [ ]:
inl format_debug forall t. (x : t) : string =
    real format_debug `t x

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda

{ c = "1"; a = "2"; b = "3" }
|> format_debug
|> _assert_eq (
    backend_switch {
        Gleam = fun () => "#(\"1\", \"2\", \"3\")" : string
        Fsharp = fun () => "struct (\"1\", \"2\", \"3\")" : string
        Python = fun () => "('1', '2', '3')" : string
    }
)

.py output (Python):
{ name = __assert_eq; expected = ('1', '2', '3') }

.gleam output (Gleam):
{ name = __assert_eq; expected = #("1", "2", "3") }



.fsx output:
{ name = __assert_eq; expected = struct ("1", "2", "3") }


### format_pretty

In [ ]:
//// real

inl format_pretty forall t. (x : t) : string =
    run_target_args `string `t (fun () => x) function
        | Rust _ => fun x =>
            open rust
            inl result = rust.emit_expr `t `std_string x ($'"format\!(\\\"{:\#?}\\\", $0)"' : string)
            from_std_string result
        | _ => fun _ => format_debug `t x

In [ ]:
inl format_pretty forall t. (x : t) : string =
    real sm'_real.format_pretty `t x

### prim

In [ ]:
inl prim x = real
    match x with
    | (x : i8) | (x : i16) | (x : i32) | (x : i64) => "%d", x
    | (x : u8) | (x : u16) | (x : u32) | (x : u64) => "%u", x
    | (x : f32) | (x : f64) => "%f", x
    | (x : string) => "%s", x
    | (x : char) => "%c", x

### printable

In [ ]:
//// real

prototype printable t : t -> ()

### format_real

In [ ]:
//// real

let format_real forall t. (x : t) : string =
    inl result = mut `string (join "")
    inl rec write x =
        inl p ((a : string), b) =
            inl s : string =
                backend_switch `string `({}) {
                    Gleam =
                        (fun () =>
                            match b with
                            | (_ : string) => $'!b ' : string
                            // | (_ : f32) | (_ : f64) => format_debug `(`b) b
                            | _ => format_debug `(`b) b
                        ) : () -> string
                    Fsharp =
                        (fun () =>
                            match b with
                            | (_ : f32) | (_ : f64) => $'$"%+.6f{!b}"' : string
                            | _ => $'$"{!b}"' : string
                        ) : () -> string
                    Python =
                        (fun () =>
                            match b with
                            | (_ : f32) | (_ : f64) => $'"{:.6f}".format(!b)' : string
                            | _ => $'f"{!b}"' : string
                        ) : () -> string
                }
            exec_unit ((fun () => result <- (++#) ((~*) `string result) s) : () -> ())

        match x with // According to Bing it shouldn't matter whether these are %d or %lld in printf.
        | () => ()
        | (x : i8) | (x : i16) | (x : i32) | (x : i64) => p ("%d", x)
        | (x : u8) | (x : u16) | (x : u32) | (x : u64) => p ("%u", x)
        | (x : f32) | (x : f64) => p ("%f", x)
        | (x : string) => p ("%s", x)
        | (x : char) => p ("%c", x)
        | (x : bool) => p ("%s", if x then "true" else "false")
        | (a,b) => write a . write ", " . write b
        | {} as x =>
            write "{ "
            inl _result =
                real_core.record_fold
                    fun { state = separator key value } =>
                        write separator
                        write (symbol_to_string `(`key)) . write " = " . write value
                        "; "
                    () x
            write " }"
        | x when real_core.symbol_is x => write (symbol_to_string `(`x))
        | x when real_core.function_is x => write (x ())
        | x when real_core.union_is x =>
            if real_core.prototype_has `(`x) printable then printable `(`x) x
            else
                write (format_debug `(`x) x)
                // real_core.unbox x (fun (k, v) =>
                //     write k
                //     match v with
                //     | () => ()
                //     | _ => write "(" . write v . write ")"
                //     )
        | x when real_core.nominal_is x =>
            if real_core.prototype_has `(`x) printable then printable `(`x) x
            // elif layout_is x then write *x // TODO: Deal with all the layout type cases.
            else write (format_pretty `(`x) x)
        | x => write (format_debug `(`x) x)
    write x
    (~*) `string result

### format

In [ ]:
inl format forall t. (x : t) : string =
    real format_real `t x

In [ ]:
//// test
////! gleam
///! fsharp
////! cuda
////! rust
////! typescript
////! python

("1", "2", ["3"; "4"], { b = "5"; c = "6"; a = fun () => "7" })
|> format
|> _assert_eq "1, 2, UH0_1 (\"3\", UH0_1 (\"4\", UH0_0)), { b = 5; c = 6; a = 7 }"

{ name = __assert_eq; expected = 1, 2, UH0_1 ("3", UH0_1 ("4", UH0_0)), { b = 5; c = 6; a = 7 } }


### to_char_array

In [ ]:
inl to_char_array (str : string) : array_base char =
    am.init (str |> length) (fun i => str |> index i)
    |> fun (a x : _ int _) => x

In [ ]:
//// test
///! fsharp
///! cuda

"abc"
|> to_char_array
|> format
|> _assert_eq (;[ 'a'; 'b'; 'c' ] |> format)

.py output (Python):
{ name = __assert_eq; expected = ['a' 'b' 'c'] }



.fsx output:
{ name = __assert_eq; expected = [|'a'; 'b'; 'c'|] }


### concat_array

In [ ]:
inl concat_array (separator : string) (input : a int string) =
    (input, { acc = ""; sep = "" })
    ||> am.foldBack fun (x : string) { acc sep } =>
            { acc = x ++# sep ++# acc; sep = separator }
    |> fun { acc } => acc

In [ ]:
//// test
///! fsharp
////! cuda // AttributeError: 'str' object has no attribute 'item'
///! rust
///! typescript
///! python
//// print_code

;[
    "1"
    "2"
    "3"
]
|> fun x =>
    inl code = (a x : _ int _) |> concat_array "\n"
    code
    |> _assert_eq "1\n2\n3"


.rs output:
{ name = __assert_eq; expected = 1
2
3 }


.ts output:
{ name = __assert_eq; expected = 1
2
3 }


.py output:
{ name = __assert_eq; expected = 1
2
3 }






.fsx:
type Mut0 = {mutable l0 : int32; mutable l1 : string; mutable l2 : string}
and Mut1 = {mutable l0 : string}
let rec method1 (v0 : int32, v1 : Mut0) : bool =
    let v2 : int32 = v1.l0
    let v3 : bool = v2 < v0
    v3
and method2 (v0 : bool) : bool =
    v0
and method4 () : string =
    let v0 : string = ""
    v0
and closure0 (v0 : Mut1, v1 : string) () : unit =
    let v2 : string = v0.l0
    let v4 : string = v2 + v1 
    v0.l0 <- v4
    ()
and method3 (v0 : string, v1 : string) : string =
    let v2 : string = method4()
    let v3 : Mut1 = {l0 = v2} : Mut1
    let v6 : string = "{ "
    let v7 : string = $"{v6}"
    let v15 : unit = ()
    let v16 : (unit -> unit) = closure0(v3, v7)
    let v17 : unit = (fun () -> v16 (); v15) ()
    let v25 : string = "name"
    let v26 : string = $"{v25}"
    let v34 : unit = ()
    let v35 : (unit -> unit) = closure0(v3, v26)
    let v36 : unit = (fun () -> v35 (); v34) ()
    let v44 : string = " = "
    let v45 : string = $"{v44}"
    l

### concat_list

In [ ]:
inl concat_list separator input =
    (input, { acc = ""; sep = "" })
    ||> listm.foldBack fun (x : string) { acc sep } => {
        acc = x ++# sep ++# acc
        sep = separator
    }
    |> fun { acc } => acc

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda
///! rust
///! typescript
///! python
//// print_code

[
    "1"
    "2"
    "3"
]
|> fun x =>
    inl code = (x : _) |> concat_list "\n"
    code
    |> _assert_eq "1\n2\n3"


.py output (Python):
{ name = __assert_eq; expected = 1
2
3 }


.rs output:
{ name = __assert_eq; expected = 1
2
3 }


.ts output:
{ name = __assert_eq; expected = 1
2
3 }


.py output:
{ name = __assert_eq; expected = 1
2
3 }


.gleam output (Gleam):
{ name = __assert_eq; expected = 1
2
3 }






.gleam (Gleam):
import gleam/io
pub type Mut0 { Mut0(l0 : String) }
pub fn method1 (v0 :    Bool) -> Bool {
    v0
}
pub fn method3 () -> String {
    let v0 = ""
    v0
}
pub fn method2 (v0 :    String, v1 :    String) -> String {
    let v2 = method3()
    let v3 = Mut0(l0 :  v2)
    let v4 = "{ "
    let v5 = v4 
    let v9 = v3.l0 
    let v10 = v9 <> v5 
    let v3 = Mut0(l0: v10)
    let v22 = "name"
    let v23 = v22 
    let v27 = v3.l0 
    let v28 = v27 <> v23 
    let v3 = Mut0(l0: v28)
    let v40 = " = "
    let v41 = v40 
    let v45 = v3.l0 
    let v46 = v45 <> v41 
    let v3 = Mut0(l0: v46)
    let v58 = v0 
    let v62 = v3.l0 
    let v63 = v62 <> v58 
    let v3 = Mut0(l0: v63)
    let v75 = "; "
    let v76 = v75 
    let v80 = v3.l0 
    let v81 = v80 <> v76 
    let v3 = Mut0(l0: v81)
    let v93 = "expected"
    let v94 = v93 
    let v98 = v3.l0 
    let v99 = v98 <> v94 
    let v3 = Mut0(l0: v99)
    let v111 = v40 
    let v115 = v3.l0 
    let v116 = v115 

### ellipsis

In [ ]:
inl ellipsis (max : i32) (s : string) =
    if sm.length s <= max
    then s
    else s |> slice 0 (max - 1) |> append "..."

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda
///! rust
///! typescript
///! python

"12345"
|> ellipsis 2
|> _assert_eq "12..."

"12345"
|> ellipsis 4
|> _assert_eq "1234..."


.py output (Python):
{ name = __assert_eq; expected = 12... }
{ name = __assert_eq; expected = 1234... }


.rs output:
{ name = __assert_eq; expected = 12... }
{ name = __assert_eq; expected = 1234... }


.ts output:
{ name = __assert_eq; expected = 12... }
{ name = __assert_eq; expected = 1234... }


.py output:
{ name = __assert_eq; expected = 12... }
{ name = __assert_eq; expected = 1234... }


.gleam output (Gleam):
{ name = __assert_eq; expected = 12... }
{ name = __assert_eq; expected = 1234... }




.fsx output:
{ name = __assert_eq; expected = 12... }
{ name = __assert_eq; expected = 1234... }


## fsharp

### last_index_of

In [ ]:
inl last_index_of (search : string) (s : string) : i32 =
    $'!s.LastIndexOf !search '

### index_of

In [ ]:
inl index_of (search : string) (s : string) : int =
    backend_switch {
        Gleam = fun () =>
            global "import gleam/list"
            global "import gleam/option"
            global "import gleam/string"
            inl g : list string =
                $'!s |> string.to_graphemes'
            $'let search = !search ' : ()
            $'let x = !g |> list.index_fold (option.None, fn(acc, item, index) {' : ()
            $'  case acc {' : ()
            $'    option.None if item == search -> option.Some(index)' : ()
            $'    _ -> acc' : ()
            $'  }' : ()
            $'})' : ()
            $'x |> option.unwrap(-1)' : int
        Fsharp = fun () => $'!s.IndexOf !search ' : int
        Python = fun () => $'!s.find(!search)' : int
    }

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda
///! rust
///! typescript
///! python

"123"
|> index_of "2"
|> _assert_eq 1

.py output (Python):
{ name = __assert_eq; expected = 1 }

.rs output:
{ name = __assert_eq; expected = 1 }

.ts output:
{ name = __assert_eq; expected = 1 }

.py output:
{ name = __assert_eq; expected = 1 }

.gleam output (Gleam):
{ name = __assert_eq; expected = 1 }



.fsx output:
{ name = __assert_eq; expected = 1 }


### replicate

In [ ]:
inl replicate (n : int) (s : string) : string =
    inl rec body i acc =
        if i >= n
        then acc
        else 루프 (i + 1) (acc ++# s)
    and inl 루프 i = join_body_unit body n i
    루프 0 ""

In [ ]:
//// test
///! gleam
///! fsharp

"12"
|> replicate (join 3)
|> _assert_eq "121212"

.gleam output (Gleam):
{ name = __assert_eq; expected = 121212 }



.fsx output:
{ name = __assert_eq; expected = 121212 }


In [ ]:
//// test
///! gleam
///! fsharp
//// print_code

"12"
|> replicate 3
|> _assert_eq "121212"

.gleam output (Gleam):
{ name = __assert_eq; expected = 121212 }





.gleam (Gleam):
import gleam/io
pub type Mut0 { Mut0(l0 : String) }
pub fn method1 (v0 :    Bool) -> Bool {
    v0
}
pub fn method3 () -> String {
    let v0 = ""
    v0
}
pub fn method2 (v0 :    String, v1 :    String) -> String {
    let v2 = method3()
    let v3 = Mut0(l0 :  v2)
    let v4 = "{ "
    let v5 = v4 
    let v9 = v3.l0 
    let v10 = v9 <> v5 
    let v3 = Mut0(l0: v10)
    let v22 = "name"
    let v23 = v22 
    let v27 = v3.l0 
    let v28 = v27 <> v23 
    let v3 = Mut0(l0: v28)
    let v40 = " = "
    let v41 = v40 
    let v45 = v3.l0 
    let v46 = v45 <> v41 
    let v3 = Mut0(l0: v46)
    let v58 = v0 
    let v62 = v3.l0 
    let v63 = v62 <> v58 
    let v3 = Mut0(l0: v63)
    let v75 = "; "
    let v76 = v75 
    let v80 = v3.l0 
    let v81 = v80 <> v76 
    let v3 = Mut0(l0: v81)
    let v93 = "expected"
    let v94 = v93 
    let v98 = v3.l0 
    let v99 = v98 <> v94 
    let v3 = Mut0(l0: v99)
    let v111 = v40 
    let v115 = v3.l0 
    let v116 = v115 

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda
///! rust
///! typescript
///! python

"12"
|> replicate 3
|> _assert_eq "121212"

.py output (Python):
{ name = __assert_eq; expected = 121212 }

.rs output:
{ name = __assert_eq; expected = 121212 }

.ts output:
{ name = __assert_eq; expected = 121212 }

.py output:
{ name = __assert_eq; expected = 121212 }

.gleam output (Gleam):
{ name = __assert_eq; expected = 121212 }



.fsx output:
{ name = __assert_eq; expected = 121212 }


### string_builder

In [ ]:
nominal string_builder_python =
    `(
        global "import io"
        $'' : $'io.StringIO'
    )
type string_builder_switch =
    {
        Gleam : $'String'
        Fsharp : $'System.Text.StringBuilder'
        Python : string_builder_python
    }
nominal string_builder = $'backend_switch `(string_builder_switch)'

inl string_builder (initial : string) : string_builder =
    inl initial =
        if initial = ""
        then join initial
        else initial

    backend_switch {
        Gleam = fun () => $'!initial ' : string_builder
        Fsharp = fun () => initial |> $'`string_builder ' : string_builder
        Python = fun () =>
            global "import io"
            global "class CustomStringIO(io.StringIO):\n    def __init__(self, init=''):\n        super().__init__()\n        if init != '': self.write(init)\n    def __str__(self): return self.getvalue()\n    def __repr__(self): return self.getvalue()"
            $'CustomStringIO(!initial)' : string_builder
    }

### obj_to_string

In [ ]:
inl obj_to_string x : string =
    backend_switch {
        Gleam = fun () =>
            real
                typecase `x with
                | string => x
                | char => to `(`x) `string x
                | string_builder => to `(`x) `string x
                | _ =>
                    global "import gleam/string"
                    $'string.inspect( !x)' : string
            : string
        Fsharp = fun () => x |> $'_.ToString()' : string
        Python = fun () => $'str(!x)' : string
    }

### pad_left

In [ ]:
inl pad_left (total_width : int) (padding_char : char) (s : string) : string =
    inl padding = padding_char |> obj_to_string |> replicate (total_width - length s)
    padding ++# s

### pad_right

In [ ]:
inl pad_right (total_width : int) (padding_char : char) (s : string) : string =
    inl padding = padding_char |> obj_to_string |> replicate (total_width - length s)
    s ++# padding

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda
///! rust
///! typescript
///! python

"123"
|> pad_right 6 ' '
|> _assert_eq "123   "

.py output (Python):
{ name = __assert_eq; expected = 123    }

.rs output:
{ name = __assert_eq; expected = 123    }

.ts output:
{ name = __assert_eq; expected = 123    }

.py output:
{ name = __assert_eq; expected = 123    }

.gleam output (Gleam):
{ name = __assert_eq; expected = 123    }



.fsx output:
{ name = __assert_eq; expected = 123    }


### convert_to_utf32

In [ ]:
inl convert_to_utf32 (c : char) : int =
    backend_switch {
        Fsharp = fun () =>
            run_target_args' c function
            | Fsharp (Native) => fun c => $'System.Char.ConvertToUtf32 (string !c, 0)' : int
            | _ => fun c => c |> i32
        Python = fun () => $'ord(!c)' : int
    }

### ends_with

In [ ]:
inl ends_with (value : string) (s : string) : bool =
    backend_switch {
        Gleam = fun () =>
            global "import gleam/string"
            $'!s |> string.ends_with(!value)' : bool
        Fsharp = fun () => $'!s.EndsWith (!value, false, null)' : bool
        Python = fun () => $'!s.endswith(!value)' : bool
    }

### starts_with

In [ ]:
inl starts_with (value : string) (s : string) : bool =
    backend_switch {
        Gleam = fun () =>
            global "import gleam/string"
            $'!s |> string.starts_with(!value)' : bool
        Fsharp = fun () => $'!s.StartsWith (!value, false, null)' : bool
        Python = fun () => $'!s.startswith(!value)' : bool
    }

### is_white_space

In [ ]:
inl is_white_space (c : char) : bool =
    c |> $'System.Char.IsWhiteSpace'

### substring

In [ ]:
inl substring (start : i32) (len : i32) (str : string) : string =
    $'!str.Substring (!start, !len)'

### to_lower

In [ ]:
inl to_lower (input : string) : string =
    backend_switch {
        Gleam = fun () =>
            global "import gleam/string"
            $'!input |> string.lowercase' : string
        Fsharp = fun () => $'!input.ToLower' () : string
        Python = fun () => $'!input.lower()' : string
    }

### to_upper

In [ ]:
inl to_upper (input : string) : string =
    $'!input.ToUpper' ()

### char_to_upper

In [ ]:
inl char_to_upper (input : char) : char =
    backend_switch {
        Gleam = fun () =>
            global "import gleam/string"
            $'!input |> string.uppercase' : char
        Fsharp = fun () => $'System.Char.ToUpper !input ' : char
        Python = fun () => $'!input.upper()' : char
    }

### builder_append

In [ ]:
inl builder_append (item : string) (sb : string_builder) : string_builder =
    backend_switch {
        Gleam = fun () =>
            $'!sb <> !item ' : string_builder
        Fsharp = fun () =>
            ($'!sb.Append' item : string_builder) |> ignore
            sb
        Python = fun () =>
            ($'!sb.write(!item)' : int) |> ignore
            sb
    }

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda
///! rust
///! typescript
///! python

"ab"
|> string_builder
|> builder_append "cd"
|> builder_append "ef"
|> obj_to_string
|> _assert_eq "abcdef"

.py output (Python):
{ name = __assert_eq; expected = abcdef }

.rs output:
{ name = __assert_eq; expected = abcdef }

.ts output:
{ name = __assert_eq; expected = abcdef }

.py output:
{ name = __assert_eq; expected = abcdef }

.gleam output (Gleam):
{ name = __assert_eq; expected = abcdef }



.fsx output:
{ name = __assert_eq; expected = abcdef }


### builder_append_line

In [ ]:
inl builder_append_line (sb : string_builder) : string_builder =
    backend_switch {
        Gleam = fun () =>
            sb |> builder_append "\n"
        Fsharp = fun () =>
            ($'!sb.AppendLine ()' : string_builder) |> ignore
            sb
        Python = fun () =>
            sb |> builder_append "\n"
    }

### builder_replace

In [ ]:
inl builder_replace (old : string) (new : string) (sb : string_builder) : string_builder =
    backend_switch {
        Gleam = fun () =>
            global "import gleam/string"
            $'!sb |> string.replace(!old, !new)' : string_builder
        Fsharp = fun () =>
            ($'!sb.Replace (!old, !new)' : string_builder) |> ignore
            sb
        Python = fun () =>
            ($'!sb.getvalue().replace(!old, !new)' : string) |> string_builder
    }

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda
///! rust
///! typescript
///! python

"ab"
|> string_builder
|> builder_append "cd"
|> builder_replace "cd" "ef"
|> builder_append "gh"
|> obj_to_string
|> _assert_eq "abefgh"

.py output (Python):
{ name = __assert_eq; expected = abefgh }

.rs output:
{ name = __assert_eq; expected = abefgh }

.ts output:
{ name = __assert_eq; expected = abefgh }

.py output:
{ name = __assert_eq; expected = abefgh }

.gleam output (Gleam):
{ name = __assert_eq; expected = abefgh }



.fsx output:
{ name = __assert_eq; expected = abefgh }


### builder_clear

In [ ]:
inl builder_clear (sb : string_builder) : string_builder =
    backend_switch {
        Gleam = fun () =>
            $'let !sb = ""'
            sb
        Fsharp = fun () =>
            ($'!sb.Clear' () : string_builder) |> ignore
            sb
        Python = fun () =>
            ($'!sb.truncate(0)' : int) |> ignore
            ($'!sb.seek(0)' : int) |> ignore
            sb
    }

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda
///! rust
///! typescript
///! python

"ab"
|> string_builder
|> builder_append "cd"
|> builder_clear
|> builder_append "gh"
|> obj_to_string
|> _assert_eq "gh"

.py output (Python):
{ name = __assert_eq; expected = gh }

.rs output:
{ name = __assert_eq; expected = gh }

.ts output:
{ name = __assert_eq; expected = gh }

.py output:
{ name = __assert_eq; expected = gh }

.gleam output (Gleam):
{ name = __assert_eq; expected = gh }



.fsx output:
{ name = __assert_eq; expected = gh }


### builder_insert

In [ ]:
inl builder_insert (i : int) (s : string) (sb : string_builder) : string_builder =
    backend_switch {
        Gleam = fun () =>
            inl sb = sb |> obj_to_string
            inl a = sb |> slice 0 i
            inl b = sb |> slice i (length sb)
            [ a; s; b ] |> concat_list "" |> string_builder
        Fsharp = fun () =>
            ($'!sb.Insert (!i, !s)' : string_builder) |> ignore
            sb
        Python = fun () =>
            inl sb = sb |> obj_to_string
            $'"".join([!sb[:!i], !s, !sb[!i:]])' |> string_builder
    }

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda
////! rust
////! typescript
////! python

"ab"
|> string_builder
|> builder_insert 1 "gh"
|> obj_to_string
|> _assert_eq "aghb"

.py output (Python):
{ name = __assert_eq; expected = aghb }

.gleam output (Gleam):
{ name = __assert_eq; expected = aghb }



.fsx output:
{ name = __assert_eq; expected = aghb }


### trim

In [ ]:
inl trim (input : string) : string =
    backend_switch {
        Gleam = fun () =>
            global "import gleam/string"
            $'!input |> string.trim' : string
        Fsharp = fun () => $'!input.Trim' () : string
        Python = fun () => $'!input.strip()' : string
    }

### concat

In [ ]:
inl concat (a : string) (b : seq.seq' string) : string =
    backend_switch {
        Gleam = fun () =>
            global "import gleam/string"
            $'!b |> string.join(!a)' : string
        Fsharp = fun () =>
            inl a =
                if a = "\n"
                then join a
                else a
            b |> $'String.concat' a : string
        Python = fun () =>
            $'!a.join(!b)' : string
    }

### length'

In [ ]:
inl length' forall dim. (input : string) : dim =
    backend_switch {
        Gleam = fun () =>
            global "import gleam/string"
            $'string.length(!input)' : dim
        Fsharp = fun () => input |> $'String.length' : dim
        Python = fun () => $'len(!input)' : dim
    }

In [ ]:
//// test
///! fsharp
///! cuda

"abc"
|> length'
|> _assert_eq 3i32

.py output (Python):
{ name = __assert_eq; expected = 3 }



.fsx output:
{ name = __assert_eq; expected = 3 }


### to_string any

In [ ]:
instance to_string any =
    obj_to_string

### (~$)

In [ ]:
inl (~$) s =
    s |> obj_to_string

### replace

In [ ]:
inl replace (old_value : string) (new_value : string) (s : string) : string =
    $'!s.Replace (!old_value, !new_value)'

### split

In [ ]:
inl split (separator : string) (str : string) : array_base string =
    backend_switch {
        Gleam = fun () =>
            global "import gleam/string"
            ($'!str |> string.split(!separator)' : listm'.list' string)
            |> listm'.to_array'
        Fsharp = fun () => $'!str.Split !separator ' : array_base string
        Python = fun () => $'!str.split(!separator)' : array_base string
    }

### split_string

In [ ]:
inl split_string (separator : array_base string) (str : string) : array_base string =
    run_target_args (fun () => str, separator) function
        | Fsharp (Native) => fun str, separator => $'!str.Split (!separator, System.StringSplitOptions.None)'
        | _ => fun str, separator => str |> split ((a separator : _ int _) |> seq.of_array |> concat (join ""))

### join'

In [ ]:
inl join' (concat : string) (s : a int string) : string =
    $'System.String.Join (!concat, !s)'

### encoding

In [ ]:
nominal encoding = $'System.Text.Encoding'

### encoding_utf8

In [ ]:
inl encoding_utf8 () : encoding =
    $'`encoding.UTF8'

### utf8_get_bytes

In [ ]:
inl utf8_get_bytes (s : string) : a i32 u8 =
    s |> (encoding_utf8 () |> $'_.GetBytes')

### byte_to_string

In [ ]:
inl byte_to_string (format : string) (x : u8) : string =
    $'!x.ToString' format

## rust

### str

In [ ]:
nominal str =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"str\")>]\ntype Str = class end\n\#else\ntype Str = string\n#endif\n"
        $'' : $'Str'
    )

### chars

In [ ]:
inl chars (x : rust.ref str) : rust.mut' (into_iterator char) =
    !\\(x, $'$"$0.chars()"')

### char_is_alphanumeric

In [ ]:
inl char_is_alphanumeric (x : char) : bool =
    !\\(x, $'$"$0.is_alphanumeric()"')

### byte_slice

In [ ]:
inl byte_slice (s : string) : rust.ref (am'.slice u8) =
    !\($'"b\\\"" + !s + "\\\""')

### display

In [ ]:
nominal display t =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::fmt::Display<$0>\")>]\n#endif\ntype std_fmt_Display<'T> = class end"
        $'' : $'std_fmt_Display<`t>'
    )

### base64_decode_error

In [ ]:
nominal base64_decode_error =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"base64::DecodeError\")>]\n#endif\ntype base64_DecodeError = class end"
        $'' : $'base64_DecodeError'
    )

### borsh_io_error

In [ ]:
nominal borsh_io_error =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"borsh::io::Error\")>]\n#endif\ntype borsh_io_Error = class end"
        $'' : $'borsh_io_Error'
    )

### utf8_error

In [ ]:
nominal utf8_error =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::str::Utf8Error\")>]\n#endif\ntype std_str_Utf8Error = class end"
        $'' : $'std_str_Utf8Error'
    )

### from_utf8_error

In [ ]:
nominal from_utf8_error =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::string::FromUtf8Error\")>]\n#endif\ntype std_string_FromUtf8Error = class end"
        $'' : $'std_string_FromUtf8Error'
    )

### json_value

In [ ]:
nominal json_value =
    `(
        backend_switch `(()) `({}) {
            Fsharp = (fun () =>
                global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"serde_json::Value\")>]\n#endif\ntype serde_json_Value = class end"
                        ) : () -> ()
        }
        $'' : $'serde_json_Value'
    )

### json_error

In [ ]:
nominal json_error =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"serde_json::Error\")>]\n#endif\ntype serde_json_Error = class end"
        $'' : $'serde_json_Error'
    )

### serde_wasm_bindgen_error

In [ ]:
nominal serde_wasm_bindgen_error =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"serde_wasm_bindgen::Error\")>]\n#endif\ntype serde_wasm_bindgen_Error = class end"
        $'' : $'serde_wasm_bindgen_Error'
    )

### js_string

In [ ]:
nominal js_string =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"js_sys::JsString\")>]\n#endif\ntype js_sys_JsString = class end"
        $'' : $'js_sys_JsString'
    )

### os_str

In [ ]:
nominal os_str =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::ffi::OsStr\")>]\n#endif\ntype std_ffi_OsStr = class end"
        $'' : $'std_ffi_OsStr'
    )

### c_str

In [ ]:
nominal c_str =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::ffi::CStr\")>]\n#endif\ntype std_ffi_CStr = class end"
        $'' : $'std_ffi_CStr'
    )

### c_string

In [ ]:
nominal c_string =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::ffi::CString\")>]\n#endif\ntype std_ffi_CString = class end"
        $'' : $'std_ffi_CString'
    )

### os_string

In [ ]:
nominal os_string =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::ffi::OsString\")>]\n#endif\ntype std_ffi_OsString = class end"
        $'' : $'std_ffi_OsString'
    )

### raw_string_literal

In [ ]:
inl raw_string_literal (s : string) : rust.ref str =
    !\($'"r\#\\"" + !s + "\\"\#"')

### raw_string_literal_static

In [ ]:
inl raw_string_literal_static (s : string) : rust.static_ref str =
    !\($'"r\#\\"" + !s + "\\"\#"')

### (~#)

In [ ]:
inl (~#) s =
    s |> raw_string_literal

### (~##)

In [ ]:
inl (~##) s =
    s |> raw_string_literal_static

### include_str

In [ ]:
inl include_str (path : string) : rust.ref str =
    !\($'"include_str\!(\\\"" + !path + "\\\")"')

### as_str

In [ ]:
inl as_str (s : string) : rust.ref str =
    // !\\(s, $'"fable_library_rust::String_::LrcStr::as_str(&$0)"')
    run_target_args (fun () => s) function
        | Rust _ => fun s => !\\(s, $'"&*$0"')
        | _ => fun s => s |> unbox

### from_iter

In [ ]:
inl from_iter forall (t : * -> *). (s : t char) : std_string =
    !\\(s, $'"String::from_iter($0)"')

### ref_to_std_string

In [ ]:
inl ref_to_std_string (str : rust.ref str) : std_string =
    run_target_args (fun () => str) function
        | Rust _ => fun str => !\\(str, $'"String::from($0)"')
        | _ => fun str => str |> unbox

### cow_to_std_string

In [ ]:
inl cow_to_std_string (str : rust.cow str) : std_string =
    !\\(str, $'"String::from($0)"')

### to_std_string

In [ ]:
inl to_std_string (s : string) : std_string =
    s |> as_str |> ref_to_std_string

### as_str_std

In [ ]:
inl as_str_std (s : std_string) : rust.ref str =
    !\\(s, $'"$0.as_str()"')

### into_boxed_str

In [ ]:
inl into_boxed_str (s : std_string) : rust.box str =
    !\\(s, $'"$0.into_boxed_str()"')

### os_string_as_ref

In [ ]:
inl os_string_as_ref (s : os_string) : rust.ref os_str =
    !\\(s, $'"$0.as_ref()"')

### to_os_string

In [ ]:
inl to_os_string (s : rust.ref os_str) : os_string =
    !\\(s, $'"$0.to_os_string()"')

### new_c_string

In [ ]:
inl new_c_string (s : std_string) : c_string =
    !\\(s, $'"std::ffi::CString::new($0).unwrap()"')

### os_to_str

In [ ]:
inl os_to_str (s : os_string) : optionm'.option' (rust.ref str) =
    !\\(s, $'"$0.to_str()"')

### from_os_str_ref

In [ ]:
inl from_os_str_ref s =
    s
    |> to_os_string
    |> os_to_str
    |> optionm'.unwrap
    |> ref_to_std_string
    |> from_std_string

### format_custom'

In [ ]:
inl format_custom' (f : string) x : std_string =
    run_target function
        | Rust _ => fun () =>
            !\\(x, $'"format\!(\\\"" + !f + "\\\", $0)"')
        | _ => fun () => null ()

### format_debug'

In [ ]:
inl format_debug' x : std_string =
    run_target function
        | Rust _ => fun () =>
            !\\(x, $'"format\!(\\\"{:?}\\\", $0)"')
        | _ => fun () => null ()

### format'

In [ ]:
inl format' x : std_string =
    run_target_args (fun () => x) function
        | Rust _ => fun x =>
            !\\(x, $'"format\!(\\\"{}\\\", $0)"')
        | _ => fun _ => null ()

### format_hex'

In [ ]:
inl format_hex' x : std_string =
    !\\(x, $'"format\!(\\\"{:02x}\\\", $0)"')

### format''

In [ ]:
inl format'' (format : string) : std_string =
    !\($'@@$"format\!(" + !format + ")"')

### regex

In [ ]:
nominal regex =
    `(
        backend_switch `(()) `({}) {
            Fsharp = (fun () =>
                global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"regex::Regex\")>]\n#endif\ntype regex_Regex = class end"
                        ) : () -> ()
        }
        $'' : $'regex_Regex'
    )

### regex_error

In [ ]:
nominal regex_error =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"regex::Error\")>]\n#endif\ntype regex_Error = class end"
        $'' : $'regex_Error'
    )

### new_regex

In [ ]:
inl new_regex (pattern : string) : resultm.result' regex regex_error =
    !\\(pattern, $'$"regex::Regex::new(&$0)"')

### captures

In [ ]:
nominal regex_captures t =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"regex::Captures<$0>\")>]\n#endif\ntype regex_Captures<'T> = class end"
        $'' : $'regex_Captures<`t>'
    )

### regex_capture_matches

In [ ]:
nominal regex_capture_matches =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"regex::CaptureMatches\")>]\n#endif\ntype regex_CaptureMatches = class end"
        $'' : $'regex_CaptureMatches'
    )

### regex_capture_names

In [ ]:
nominal regex_capture_names =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"regex::CaptureNames\")>]\n#endif\ntype regex_CaptureNames = class end"
        $'' : $'regex_CaptureNames'
    )

inl regex_capture_names (regex : regex) : regex_capture_names =
    !\\(regex, $'$"$0.capture_names()"')

### match'

In [ ]:
nominal match' =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"regex::Match\")>]\n#endif\ntype regex_Match = class end"
        $'' : $'regex_Match'
    )

### regex_captures_iter

In [ ]:
inl regex_captures_iter (s : rust.static_ref (rust.mut' std_string)) (regex : regex) : regex_capture_matches =
    inl regex = regex |> rust.emit
    !\\(regex, $'$"$0.captures_iter(!s)"')

### regex_captures

In [ ]:
let regex_captures (s : string) (regex : regex) : am'.vec (mapm.hash_map string string) =
    // inl s = join s
    // !\\(regex, $'$"$0.captures_iter(&*!s).map(|caps| $0.capture_names().map(|x| x.and_then(|n| Some((n, caps.name(n)?.as_str())))).flatten().collect()).collect()"')

    inl s = s |> to_std_string
    fun () =>
        inl matches =
            inl s = s |> rust.new_box |> rust.box_leak
            regex |> regex_captures_iter s

        (!\($'"true; let _regex_captures : Vec<_> = !matches.map(|x| { //"') : bool) |> ignore

        inl fn (match' : rust.static_ref (rust.mut' (regex_captures rust.static_lifetime))) : mapm.hash_map string string =

            inl names = regex |> regex_capture_names

            (!\($'"true; let _regex_captures : std::collections::HashMap<_, _> = !names.map(|x| { //"') : bool) |> ignore

            inl fn (n : string) : pair string string =
                inl n' = n |> rust.clone

                new_pair n' !\\(n, $'$"!match'.name(&$0).map(|x| x.as_str()).unwrap_or(\\\"\\\").to_string().into()"')

            (!\\(fn !\($'"x.unwrap_or(\\\"\\\").to_string().into()"'), $'"true; $0 }).map(|x| std::sync::Arc::try_unwrap(x).unwrap_or_else(|x| (*x).clone())).collect()"') : bool) |> ignore

            !\($'"_regex_captures"')

        inl x =
            !\($'$"x"')
            |> rust.new_box
            |> rust.box_leak

        (!\\(fn x, $'"true; $0 }).collect::<Vec<_>>()"') : bool) |> ignore

        !\($'"_regex_captures"')

    |> rust.capture_move

In [ ]:
//// test
///! rust -d regex

"fable-library-ts\\.(?<a>[\\d.]+)$"
|> new_regex
|> resultm.unwrap'
|> regex_captures "fable_modules/fable-library-ts.4.17.0"
|> am'.vec_map (mapm.to_vec >> am'.vec_sort_by_key id)
|> format_debug
|> _assert_eq (
    ;[
        ;[ "", ""; "a", "4.17.0" ]
        |> am'.to_vec
    ]
    |> am'.to_vec
    |> format_debug
)

{ name = __assert_eq; expected = [[("", ""), ("a", "4.17.0")]] }


### replace_regex'

In [ ]:
inl replace_regex' (pattern : string) (replacement : a i32 string) (s : string) : string =
    run_target_args (fun () => s, pattern, replacement) function
        | Rust (Native) => fun s, pattern, replacement =>
            inl regex = pattern |> new_regex |> resultm.unwrap'
            !\\((regex, #s, replacement), $'$"$0.replace_all($1, &*$2)"')
            |> cow_to_std_string
            |> from_std_string
        | _ => fun _ => join ""

### serialize

In [ ]:
inl serialize forall t. (x : t) : resultm.result' std_string json_error =
    !\($'"serde_json::to_string(&!x)"')

### deserialize

In [ ]:
inl deserialize forall t. (json : string) : resultm.result' t std_string =
    inl json = join json
    inl json = json |> as_str
    !\\(json, $'"serde_json::from_str(&$0)"')
    |> resultm.map_error' fun (x : json_error) => x |> format'

### borsh_serialize

In [ ]:
inl borsh_serialize forall t. (data : t) : am'.vec u8 =
    (!\($'"true; let mut data = Vec::new()"') : bool) |> ignore
    (!\\(data, $'"true; borsh::BorshSerialize::serialize(&$0, &mut data).unwrap()"') : bool) |> ignore
    !\($'"data"')

### borsh_deserialize

In [ ]:
inl borsh_deserialize forall t. (data : array_base u8) : resultm.result' t std_string =
    inl data = data |> am'.as_slice
    (!\($'"true; let mut !data = !data"') : bool) |> ignore
    inl result = !\($'"borsh::BorshDeserialize::deserialize(&mut !data)"')
    result
    |> resultm.map_error' fun (x : borsh_io_error) => x |> format'

### deserialize_vec

In [ ]:
inl deserialize_vec (value : json_value) : resultm.result' (am'.vec u8) std_string =
    inl value = join value
    !\($'"serde_json::from_value(!value)"')
    |> resultm.map_error' fun (x : json_error) => x |> format'

### encode_uri_component

In [ ]:
inl encode_uri_component (s : std_string) : js_string =
    !\($'"js_sys::encode_uri_component(&!s)"')

### strip_prefix

In [ ]:
inl strip_prefix (prefix : char) (s : std_string) : optionm'.option' (rust.ref str) =
    inl s = join s
    !\($'"!s.strip_prefix(!prefix)"')

### str_from_utf8

In [ ]:
inl str_from_utf8 (bytes : rust.ref (am'.slice u8)) : resultm.result' (rust.ref str) utf8_error =
    !\\(bytes, $'"std::str::from_utf8($0)"')

### string_from_utf8

In [ ]:
inl string_from_utf8 (bytes : am'.vec u8) : resultm.result' std_string from_utf8_error =
    inl bytes = join bytes
    !\\(bytes, $'"std::string::String::from_utf8($0)"')

### base64_decode

In [ ]:
inl base64_decode (s : std_string) : result std_string std_string =
    fun () =>
        inl s = join s
        inl bytes : resultm.result' (am'.vec u8) base64_decode_error =
            !\($'"base64::Engine::decode(&base64::engine::general_purpose::STANDARD, !s)"')
        bytes
        |> resultm.map_error' format'
        |> resultm.try'
        |> string_from_utf8
        |> resultm.map_error' format'
    |> fun x =>
        join x ()
        |> resultm.unbox

### encoding'

In [ ]:
nominal encoding' =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"encoding_rs::Encoding\")>]\n#endif\ntype encoding_rs_Encoding = class end"
        $'' : $'encoding_rs_Encoding'
    )

### encoding_utf8'

In [ ]:
inl encoding_utf8' () : rust.ref encoding' =
    !\($'"encoding_rs::UTF_8"')

### encoding_1252

In [ ]:
inl encoding_1252' () : rust.ref encoding' =
    !\($'"encoding_rs::WINDOWS_1252"')

### encoding_encode

In [ ]:
inl encoding_encode' (encoding : rust.ref encoding') (text : string) : rust.cow (am'.slice u8) =
    !\\((encoding, text), $'"$0.encode(&*$1).0"')

### utf8_decode

In [ ]:
inl utf8_decode (data : am'.vec u8) : resultm.result' std_string (rust.cow str) =
    !\($'$"encoding::Encoding::decode(encoding::all::UTF_8, &!data, encoding::DecoderTrap::Replace)"')

### windows

In [ ]:
nominal windows t =
    `(
        global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::slice::Windows<$0>\")>]\n#endif\ntype std_slice_Windows<'T> = class end"
        $'' : $'std_slice_Windows<`t>'
    )

inl windows (len : unativeint) (source : am'.vec u8) : windows u8 =
    inl source = source |> rust.new_box |> rust.box_leak
    // inl source' = source |> rust.clone
    inl result = !\\(len, $'"<[_]>::windows(!source, $0)"')
    // source |> rust.drop
    result

### any

In [ ]:
inl any forall t. (fn : string -> bool) (source : windows t) : bool =
    (!\($'"true; let mut !source = !source"') : bool) |> ignore
    inl fn' x =
        x
        |> str_from_utf8
        |> resultm.unwrap_or' #""
        |> ref_to_std_string
        |> from_std_string
        |> fn
    !\\(fn', $'"!source.any(move |x| $0(x))"')

### slice_contains

In [ ]:
inl slice_contains (text : string) (source : am'.vec u8) : bool =
    fun () =>
        inl source = join source
        source
        |> windows (text |> length |> (fun x => x : i32) |> convert)
        |> any ((=.) text)
    |> fun x => join x ()

### as_bytes

In [ ]:
inl as_bytes (text : string) : rust.ref (am'.slice u8) =
    inl text = join text
    !\($'"!text.as_bytes()"')

### into_bytes

In [ ]:
inl into_bytes (x : std_string) : am'.vec u8 =
    !\\(x, $'$"$0.into_bytes()"')

## python

### encode_utf8

In [ ]:
inl encode_utf8 (s : string) : string =
    inl encoding = "utf-8"
    backend_switch {
        Fsharp = fun () =>
            open python_operators
            !\\((s, encoding), $'"$0.encode($1)"') : string
        Python = fun () =>
            $'!s.encode(!encoding)' : string
    }

## sm'

### contains

In [ ]:
inl contains (value : string) (s : string) : bool =
    backend_switch {
        Gleam = fun () =>
            global "import gleam/string"
            $'!s |> string.contains(!value)' : bool
        Fsharp = fun () => $'!s.Contains !value ' : bool
        Python = fun () => $'!value in !s ' : bool
    }

### to_string result t u

In [ ]:
instance to_string result t u = fun x =>
    real
        open rust
        typecase (t * u) with
        | string * string =>
            match x with
            | Ok x => x
            | Error x => $'"sm\'.to_string result / Error: " + !x + ""' : string
        | std_string * std_string =>
            match x with
            | Ok x => from_std_string x
            | Error x => $'"sm\'.to_string result / Error: " + string !x + ""' : string
        | _ => obj_to_string `u x

### format_exception

In [ ]:
inl format_exception (ex : exn) : string =
    run_target function
        | Fsharp (Native) => fun () => $'$"{!ex.GetType ()}: {!ex.Message}"'
        | _ => fun () => ex |> format_debug

In [ ]:
//// test
///! fsharp
///! cuda
///! rust
///! typescript
///! python

fun () => failwith "test"
|> _throws
|> optionm.value
|> format_exception
|> _assert_eq (run_target function
    | Fsharp _ => fun () => join "System.Exception: test"
    | Cuda _ => fun () => "test"
    | Rust _ => fun () => "Exception { message: \"test\" }"
    | TypeScript _ => fun () => "Error: test"
    | Python _ => fun () => join "test"
    | _ => fun () => null ()
)

.py output (Python):
{ name = __assert_eq; expected = test }

.rs output:
{ name = __assert_eq; expected = Exception { message: "test" } }

.ts output:
{ name = __assert_eq; expected = Error: test }

.py output:
{ name = __assert_eq; expected = test }



.fsx output:
{ name = __assert_eq; expected = System.Exception: test }


### concat_list

In [ ]:
inl concat_list s list =
    list
    |> listm'.box
    |> seq.of_list'
    |> concat s

### range

In [ ]:
inl range forall t. (start : am'.range t) (end : am'.range t) s =
    inl start, end =
        inl len () =
            s |> length'
        match start, end with
        | Start start, End fn => start, len |> fn
        | End start_fn, End end_fn => start_fn len, end_fn len
    inl end' =
        backend_switch {
            Gleam = fun () =>
                // [
                    // "sm'.range"
                    // " / s: "; s
                    // " / s |> var_is: "; s |> var_is |> obj_to_string
                    // " / start |> var_is: "; start |> var_is |> obj_to_string
                    // " / end |> var_is: "; end |> var_is |> obj_to_string
                // ]
                // |> concat_list ""
                // |> console.write_line
                if
                    (end |> var_is |> not)
                    && (start |> var_is |> not)
                    && (s |> var_is)
                    || (start |> var_is |> not) && (end |> var_is)
                then 0i32 else 1
            Fsharp = fun () => 1i32
            Python = fun () => 1i32
        }
    s |> slice (start |> i32) ((end |> i32) - end')

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda
///! rust
///! typescript
///! python

"abcde"
|> range (am'.Start 0i32) (am'.End eval)
|> _assert_eq "abcde"

(join "abcde")
|> range (am'.Start 1i32) (am'.End eval)
|> _assert_eq "bcde"

(join "abcde")
|> range (am'.Start 1i32) (am'.End fun x => 5)
|> _assert_eq "bcde"

"abcde"
|> range (am'.End fun x => x () - 4i32) (am'.End fun x => x () - 1)
|> _assert_eq "bcd"

(join "abcde")
|> range (am'.End fun x => x () - 4i32) (am'.End fun x => x () - 1)
|> _assert_eq "bcd"

(join "abcde")
|> range (am'.End fun x => x () - 4i32) (am'.End fun x => 4)
|> _assert_eq "bcd"


.py output (Python):
{ name = __assert_eq; expected = abcde }
{ name = __assert_eq; expected = bcde }
{ name = __assert_eq; expected = bcde }
{ name = __assert_eq; expected = bcd }
{ name = __assert_eq; expected = bcd }
{ name = __assert_eq; expected = bcd }


.rs output:
{ name = __assert_eq; expected = abcde }
{ name = __assert_eq; expected = bcde }
{ name = __assert_eq; expected = bcde }
{ name = __assert_eq; expected = bcd }
{ name = __assert_eq; expected = bcd }
{ name = __assert_eq; expected = bcd }


.ts output:
{ name = __assert_eq; expected = abcde }
{ name = __assert_eq; expected = bcde }
{ name = __assert_eq; expected = bcde }
{ name = __assert_eq; expected = bcd }
{ name = __assert_eq; expected = bcd }
{ name = __assert_eq; expected = bcd }


.py output:
{ name = __assert_eq; expected = abcde }
{ name = __assert_eq; expected = bcde }
{ name = __assert_eq; expected = bcde }
{ name = __assert_eq; expected = bcd }
{ name = __assert_eq; expected = bcd }
{ name = __assert_eq; e

.fsx output:
{ name = __assert_eq; expected = abcde }
{ name = __assert_eq; expected = bcde }
{ name = __assert_eq; expected = bcde }
{ name = __assert_eq; expected = bcd }
{ name = __assert_eq; expected = bcd }
{ name = __assert_eq; expected = bcd }


In [ ]:
//// test
///! gleam
///! fsharp
///! cuda

"abcde"
|> range (am'.Start 1i32) (am'.End eval)
|> _assert_eq "bcde"

"abcde"
|> range (am'.End fun x => x () - 4i32) (am'.End fun x => x () - 1)
|> _assert_eq "bcd"

(join "abcde")
|> range (am'.Start 1i32) (am'.End eval)
|> _assert_eq "bcde"

(join "abcde")
|> range (am'.End fun x => x () - 4i32) (am'.End fun x => x () - 1)
|> _assert_eq "bcd"


.py output (Python):
{ name = __assert_eq; expected = bcde }
{ name = __assert_eq; expected = bcd }
{ name = __assert_eq; expected = bcde }
{ name = __assert_eq; expected = bcd }


.gleam output (Gleam):
{ name = __assert_eq; expected = bcde }
{ name = __assert_eq; expected = bcd }
{ name = __assert_eq; expected = bcde }
{ name = __assert_eq; expected = bcd }




.fsx output:
{ name = __assert_eq; expected = bcde }
{ name = __assert_eq; expected = bcd }
{ name = __assert_eq; expected = bcde }
{ name = __assert_eq; expected = bcd }


### trim_end

In [ ]:
inl trim_end (trim_chars : list char) (input : string) : string =
    inl trim_chars' = trim_chars |> listm'.box
    backend_switch {
        Gleam = fun () =>
            let rec 루프 acc =
                inl changed, result =
                    ((false, acc), trim_chars)
                    ||> listm.fold fun (changed, acc) x =>
                        if acc |> ends_with $x
                        then true, acc |> range (am'.Start 0i32) (am'.End fun x => x () - 1)
                        else changed, acc
                if changed
                    then 루프 result
                    else result
            루프 input
        Fsharp = fun () =>
            inl trim_chars' = trim_chars' |> listm'.to_array'
            $'!input.TrimEnd !trim_chars' ' : string
        Python = fun () =>
            inl trim_chars' = trim_chars' |> listm'.map obj_to_string |> seq.of_list' |> concat ""
            $'!input.rstrip(!trim_chars')' : string
    }

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda

"abcde"
|> trim_end [ ' ' ]
|> _assert_eq "abcde"

"abcde"
|> trim_end [ 'e' ]
|> _assert_eq "abcd"

"abcde"
|> trim_end [ 'e'; 'd' ]
|> _assert_eq "abc"

"abcde"
|> trim_end [ 'e'; 'd'; 'c' ]
|> _assert_eq "ab"


.py output (Python):
{ name = __assert_eq; expected = abcde }
{ name = __assert_eq; expected = abcd }
{ name = __assert_eq; expected = abc }
{ name = __assert_eq; expected = ab }


.gleam output (Gleam):
{ name = __assert_eq; expected = abcde }
{ name = __assert_eq; expected = abcd }
{ name = __assert_eq; expected = abc }
{ name = __assert_eq; expected = ab }




.fsx output:
{ name = __assert_eq; expected = abcde }
{ name = __assert_eq; expected = abcd }
{ name = __assert_eq; expected = abc }
{ name = __assert_eq; expected = ab }


### trim_start

In [ ]:
inl trim_start (trim_chars : list char) (input : string) : string =
    inl trim_chars' = trim_chars |> listm'.box
    backend_switch {
        Gleam = fun () =>
            let rec 루프 acc =
                inl changed, result =
                    ((false, acc), trim_chars)
                    ||> listm.fold fun (changed, acc) x =>
                        if acc |> starts_with $x
                        then true, acc |> range (am'.Start 1i32) (am'.End eval)
                        else changed, acc
                if changed
                    then 루프 result
                    else result
            루프 input
        Fsharp = fun () =>
            inl trim_chars' = trim_chars' |> listm'.to_array'
            $'!input.TrimStart !trim_chars' ' : string
        Python = fun () =>
            inl trim_chars' = trim_chars' |> listm'.map obj_to_string |> seq.of_list' |> concat ""
            $'!input.lstrip(!trim_chars')' : string
    }

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda

"abcde"
|> trim_start [ ' ' ]
|> _assert_eq "abcde"

"abcde"
|> trim_start [ 'a' ]
|> _assert_eq "bcde"

"abcde"
|> trim_start [ 'b'; 'a' ]
|> _assert_eq "cde"

"abcde"
|> trim_start [ 'c'; 'b'; 'a' ]
|> _assert_eq "de"


.py output (Python):
{ name = __assert_eq; expected = abcde }
{ name = __assert_eq; expected = bcde }
{ name = __assert_eq; expected = cde }
{ name = __assert_eq; expected = de }


.gleam output (Gleam):
{ name = __assert_eq; expected = abcde }
{ name = __assert_eq; expected = bcde }
{ name = __assert_eq; expected = cde }
{ name = __assert_eq; expected = de }




.fsx output:
{ name = __assert_eq; expected = abcde }
{ name = __assert_eq; expected = bcde }
{ name = __assert_eq; expected = cde }
{ name = __assert_eq; expected = de }


### ellipsis_end

In [ ]:
let ellipsis_end (max : i64) (s : string) =
    inl len = sm.length s
    if len <= max
    then s
    else
        inl half = f64 max / 2
        inl start_half = half |> math.ceil |> i64
        inl end_half = half |> math.floor |> i64
        inl start = s |> slice 0 (start_half - 1)
        inl end = s |> slice (len - end_half) (len - 1)
        (a ;[ start; "..."; end ] : _ int _)
        |> seq.of_array
        |> concat ""

In [ ]:
//// test

"12345"
|> ellipsis_end 2
|> _assert_eq "1...5"

"12345"
|> ellipsis_end 3
|> _assert_eq "12...5"

"1234567"
|> ellipsis_end 4
|> _assert_eq "12...67"

{ name = __assert_eq; expected = 1...5 }
{ name = __assert_eq; expected = 12...5 }
{ name = __assert_eq; expected = 12...67 }


### format_ellipsis

In [ ]:
inl format_ellipsis s =
    s
    |> format_debug
    |> ellipsis_end 400

### replace_regex

In [ ]:
let replace_regex (pattern : string) (replacement : string) (s : string) : string =
    run_target_args (fun () => s, pattern, replacement) function
        | Fsharp (Native) => fun s, pattern, replacement =>
            $'System.Text.RegularExpressions.Regex.Replace (!s, !pattern, !replacement)'
        | Rust (Native) => fun s, pattern, replacement =>
            inl regex = pattern |> new_regex |> resultm.unwrap'
            inl s = join s
            !\\((regex, s, replacement), $'$"$0.replace_all(&*$1, &*$2)"')
            |> cow_to_std_string
            |> from_std_string
        | _ => fun _ => join ""

In [ ]:
//// test
///! fsharp
///! rust -d regex

" 123"
|> replace_regex "\\s\\w2" ""
|> _assert_eq "3"

.rs output (rust -d regex):
{ name = __assert_eq; expected = 3 }



.fsx output:
{ name = __assert_eq; expected = 3 }


In [ ]:
//// test
///! rust -d regex

"    let main args =\n        ()\n"
|> replace_regex $'@@"(?P<a> *)(?P<b>let\\s+main\\s+.*?\\s*=)"' "$a[<EntryPoint>]\n$a$b"
|> _assert_eq "    [<EntryPoint>]\n    let main args =\n        ()\n"

{ name = __assert_eq; expected =     [<EntryPoint>]
    let main args =
        ()
 }


## main

In [ ]:
inl main () =
    $'let contains x = !contains x' : ()
    $'let ends_with x = !ends_with x' : ()
    $'let pad_left x = !pad_left x' : ()
    $'let pad_right x = !pad_right x' : ()
    $'let replace x = !replace x' : ()
    $'let replace_regex x = !replace_regex x' : ()
    inl slice (a : i32) (b : i32) c = slice a b c
    $'let slice x = !slice x' : ()
    $'let split x = !split x' : ()
    $'let split_string x = !split_string x' : ()
    $'let starts_with x = !starts_with x' : ()
    $'let substring x = !substring x' : ()
    $'let to_lower x = !to_lower x' : ()
    $'let to_upper x = !to_upper x' : ()
    $'let trim x = !trim x' : ()
    inl trim_end x = (a x : _ int _) |> am'.to_list' |> listm'.unbox |> trim_end
    $'let trim_end x = !trim_end x' : ()
    inl trim_start x = (a x : _ int _) |> am'.to_list' |> listm'.unbox |> trim_start
    $'let trim_start x = !trim_start x' : ()
    $'let ellipsis x = !ellipsis x' : ()
    $'let ellipsis_end x = !ellipsis_end x' : ()
    $'let format_exception x = !format_exception x' : ()
    $'let concat_array x = !concat_array x' : ()
    inl concat a (b : seq.seq' string) = concat a b
    $'let concat x = !concat x' : ()
    $'let join\' x = !join' x' : ()
    $'let to_char_array x = !to_char_array x' : ()

## rust

### to_string std_string

In [ ]:
open rust
instance to_string std_string = from_std_string